# Objective

#### To provide a basic overview of creating pivot tables in Python

In [1]:
# Load packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os as os

In [2]:
# Get current working directory

os.getcwd()

'/Users/Patrick/Desktop'

In [263]:
# Change the working directory

os.chdir('/Users/Patrick/Desktop')

In [264]:
# Load the 2014 - 2018 NHL Scores data

nhl = pd.read_csv('2014-2018 NHL Scores.csv')

In [265]:
# Look at the first few rows

nhl.head()

,Season,Date,Visitor,G,Home,G.1,Unnamed: 6,Att.,LOG,Notes
0,2014-2015,10/8/14,Philadelphia Flyers,1,Boston Bruins,2,NaN,17565,NaN,NaN
1,2014-2015,10/8/14,Vancouver Canucks,4,Calgary Flames,2,NaN,19289,NaN,NaN
2,2014-2015,10/8/14,San Jose Sharks,4,Los Angeles Kings,0,NaN,18514,NaN,NaN
3,2014-2015,10/8/14,Montreal Canadiens,4,Toronto Maple Leafs,3,NaN,19745,NaN,NaN
4,2014-2015,10/9/14,Winnipeg Jets,6,Arizona Coyotes,2,NaN,17125,NaN,NaN


> The "G" column is repeated so the second time it gets the name "G.1". Let's change those two column names to reflect the vistor goals and the home goals, respectively.

In [266]:
nhl = nhl.rename(columns = {'G': 'Visitor_G', 'G.1': 'Home_G'})

### Create a point differential column

In [267]:
nhl['Point_Diff'] = nhl['Home_G'] - nhl['Visitor_G']

In [268]:
nhl.head()

,Season,Date,Visitor,Visitor_G,Home,Home_G,Unnamed: 6,Att.,LOG,Notes,Point_Diff
0,2014-2015,10/8/14,Philadelphia Flyers,1,Boston Bruins,2,NaN,17565,NaN,NaN,1
1,2014-2015,10/8/14,Vancouver Canucks,4,Calgary Flames,2,NaN,19289,NaN,NaN,-2
2,2014-2015,10/8/14,San Jose Sharks,4,Los Angeles Kings,0,NaN,18514,NaN,NaN,-4
3,2014-2015,10/8/14,Montreal Canadiens,4,Toronto Maple Leafs,3,NaN,19745,NaN,NaN,-1
4,2014-2015,10/9/14,Winnipeg Jets,6,Arizona Coyotes,2,NaN,17125,NaN,NaN,-4


### Create a column identifying whether the home or visitor team won

In [269]:
conditions = [
    (nhl['Home_G'] > nhl['Visitor_G']),
    (nhl['Home_G'] < nhl['Visitor_G']),
    (nhl['Home_G'] == nhl['Visitor_G'])]

choices = ['home', 'visitor', 'tie']

In [270]:
nhl['winner'] = np.select(conditions, choices, default = 'null')

In [271]:
nhl.head()

,Season,Date,Visitor,Visitor_G,Home,Home_G,Unnamed: 6,Att.,LOG,Notes,Point_Diff,winner
0,2014-2015,10/8/14,Philadelphia Flyers,1,Boston Bruins,2,NaN,17565,NaN,NaN,1,home
1,2014-2015,10/8/14,Vancouver Canucks,4,Calgary Flames,2,NaN,19289,NaN,NaN,-2,visitor
2,2014-2015,10/8/14,San Jose Sharks,4,Los Angeles Kings,0,NaN,18514,NaN,NaN,-4,visitor
3,2014-2015,10/8/14,Montreal Canadiens,4,Toronto Maple Leafs,3,NaN,19745,NaN,NaN,-1,visitor
4,2014-2015,10/9/14,Winnipeg Jets,6,Arizona Coyotes,2,NaN,17125,NaN,NaN,-4,visitor


### Create a column listing the winning and losing teams

In [272]:
cond_for_winner = [
    (nhl['winner'] == 'home'),
    (nhl['winner'] == 'visitor'),
    (nhl['winner'] == 'tie')]

choices_for_winner = [nhl['Home'], nhl['Visitor'], 'tie']
choices_for_loser = [nhl['Visitor'], nhl['Home'], 'tie']

In [273]:
nhl['winning_team'] = np.select(cond_for_winner, choices_for_winner, default = 'null')
nhl['losing_team'] = np.select(cond_for_winner, choices_for_loser, default = 'null')

In [274]:
nhl.head()

,Season,Date,Visitor,Visitor_G,Home,Home_G,Unnamed: 6,Att.,LOG,Notes,Point_Diff,winner,winning_team,losing_team
0,2014-2015,10/8/14,Philadelphia Flyers,1,Boston Bruins,2,NaN,17565,NaN,NaN,1,home,Boston Bruins,Philadelphia Flyers
1,2014-2015,10/8/14,Vancouver Canucks,4,Calgary Flames,2,NaN,19289,NaN,NaN,-2,visitor,Vancouver Canucks,Calgary Flames
2,2014-2015,10/8/14,San Jose Sharks,4,Los Angeles Kings,0,NaN,18514,NaN,NaN,-4,visitor,San Jose Sharks,Los Angeles Kings
3,2014-2015,10/8/14,Montreal Canadiens,4,Toronto Maple Leafs,3,NaN,19745,NaN,NaN,-1,visitor,Montreal Canadiens,Toronto Maple Leafs
4,2014-2015,10/9/14,Winnipeg Jets,6,Arizona Coyotes,2,NaN,17125,NaN,NaN,-4,visitor,Winnipeg Jets,Arizona Coyotes


## Pivot Tables

### Get the average point differential by season

In [275]:
nhl.groupby('Season').mean()['Point_Diff']

Season
2014-2015    0.244715
2015-2016    0.206504
2016-2017    0.316260
2017-2018    0.313139
2018-2019    0.305271
Name: Point_Diff, dtype: float64

##### That doesn't look as nice. Let's make it into a pivot table

In [276]:
nhl.pivot_table(values = 'Point_Diff', index = 'Season', aggfunc = np.mean)

,Point_Diff
Season,
2014-2015,0.244715
2015-2016,0.206504
2016-2017,0.316260
2017-2018,0.313139
2018-2019,0.305271


#### How many times did the home team win versus the away team?

In [277]:
nhl.groupby(['winner']).size()

winner
home       3403
visitor    2829
dtype: int64

#### Now make it a probability by dividing the results by the number of rows in the data

In [278]:
nhl.groupby(['winner']).size() / len(nhl.index)

winner
home       0.546053
visitor    0.453947
dtype: float64

#### What is the percent of home team's winning in each season?

In [279]:
season_totals = nhl.groupby(['Season', 'winner']).size().unstack()
season_totals['Home_Win_Pct'] = season_totals['home'] / (season_totals['home'] + season_totals['visitor'])
season_totals['Visitor_Win_Pct'] = season_totals['visitor'] / (season_totals['home'] + season_totals['visitor'])
season_totals

winner,home,visitor,Home_Win_Pct,Visitor_Win_Pct
Season,,,,
2014-2015,666,564,0.541463,0.458537
2015-2016,651,579,0.529268,0.470732
2016-2017,688,542,0.559350,0.440650
2017-2018,716,555,0.563336,0.436664
2018-2019,682,589,0.536585,0.463415


## What is each team's win percentage over the seasons in the data?

In [280]:
team_win_totals = nhl.groupby(['winning_team']).size()
team_loss_totals = nhl.groupby(['losing_team']).size()

team_win_totals = pd.DataFrame(team_win_totals, columns = ['wins'])
team_loss_totals = pd.DataFrame(team_loss_totals, columns = ['losses'])
team_perf = team_win_totals.join(team_loss_totals)

team_perf['Win_Pct'] = team_perf['wins'] / (team_perf['wins'] + team_perf['losses'])
team_perf.sort_values(by = ['Win_Pct'], ascending = False)

,wins,losses,Win_Pct
winning_team,,,
Tampa Bay Lightning,254,156,0.619512
Washington Capitals,253,157,0.617073
Vegas Golden Knights,94,70,0.573171
St. Louis Blues,235,175,0.573171
Pittsburgh Penguins,232,178,0.565854
Nashville Predators,229,181,0.558537
Boston Bruins,226,184,0.551220
San Jose Sharks,223,187,0.543902
Anaheim Ducks,222,188,0.541463
